Scoring/Conceding in next 10 actions model - Model Tuning and Building - GBM

In [2]:
import pandas as pd
pd.options.display.max_columns = None

from expected_vaep_model.config import exp_vaep_chain_output_path, player_stats_file_path, exp_vaep_player_stats_output_path
from expected_vaep_model.models.predict_player_stats import predict_player_stats

import warnings
warnings.filterwarnings("ignore")    

In [4]:
chains = pd.read_csv(exp_vaep_chain_output_path)
player_stats = pd.read_csv(player_stats_file_path)

In [5]:
def get_player_totals_by_match(chains):
    return chains.groupby(['Match_ID', 'Player', 'Team'])[['xScore', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum().reset_index().sort_values(by=['Match_ID', "Player", "Team"])

player_value = get_player_totals_by_match(chains)
player_stats = player_stats.merge(player_value, how = "left", on = ['Match_ID', 'Player', 'Team'])

def get_receiver_totals_by_match(chains):
    return chains.groupby(['Match_ID', 'Receiver'])[['exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum().reset_index().sort_values(by=['Match_ID', "Receiver"])

chains['Receiver'] = chains['Player'].shift(-1)
receiver_value = get_receiver_totals_by_match(chains)
receiver_value = receiver_value.rename(columns = {
    "Receiver":"Player",
    "exp_vaep_value":"exp_vaep_value_received"})
player_stats = player_stats.merge(receiver_value[['Match_ID', 'Player', 'exp_vaep_value_received']], how = "left", on = ['Match_ID', 'Player'])


In [8]:
chains[['Player', 'Receiver', 'Description']].head()

,Player,Receiver,Description
0,NaN,Dayne Zorko,Centre Bounce
1,Dayne Zorko,Dayne Zorko,Hard Ball Get
2,Dayne Zorko,Oliver Florent,Handball
3,Oliver Florent,Oliver Florent,Loose Ball Get
4,Oliver Florent,George Hewett,Handball


In [6]:
player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,exp_vaep_value,exp_offensive_value,exp_defensive_value,exp_vaep_value_received
28950,2023F4_Collingwood_BrisbaneLions,Collingwood,Patrick Lipinski,2023F4,Patrick_Lipinski,MEDIUM_FORWARD,CD_I1003130,25,190,86,1,RIGHT,VIC,2016.0,2017.0,Eltham (Vic)/Northern U18/Western Bulldogs,28.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-07-17,58,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,8,NaN,NaN,NaN,NaN,81.3,16,59,NaN,NaN,NaN,1,0,0.0,0,0,NaN,12,0,NaN,NaN,NaN,2,NaN,1,NaN,NaN,NaN,4,NaN,2,0,NaN,134.0,0,NaN,NaN,0,NaN,4,NaN,1,NaN,3,5,1,3,16,2,8,NaN,NaN,Substitute,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0,1.708957,2.242900,2.910316,-0.667416,6.364760
28951,2023F4_Collingwood_BrisbaneLions,Collingwood,Scott Pendlebury,2023F4,Scott_Pendlebury,MIDFIELDER,CD_I260257,35,191,86,10,LEFT,VIC,2005.0,2006.0,Sale (Vic)/Gippsland U18,5.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1988-01-07,77,0,0.0,NaN,4,3,NaN,NaN,NaN,0,NaN,9,NaN,NaN,NaN,NaN,75.0,24,102,NaN,NaN,NaN,0,1,100.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,1,NaN,NaN,NaN,16,NaN,5,1,NaN,324.0,2,NaN,NaN,1,NaN,7,NaN,1,NaN,2,4,0,6,22,2,13,NaN,NaN,Centre,FINAL_TEAM,Centre,Midfield,NaN,NaN,2023.0,4.018784,6.947821,8.637864,-1.690042,11.783506
28952,2023F4_Collingwood_BrisbaneLions,Collingwood,Steele Sidebottom,2023F4,Steele_Sidebottom,MIDFIELDER,CD_I280965,32,184,86,22,RIGHT,VIC,2008.0,2009.0,Congupna (Vic)/Murray U18,11.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1991-01-02,86,0,1.0,NaN,1,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,75.0,20,98,NaN,NaN,NaN,1,0,50.0,0,1,NaN,8,0,NaN,NaN,NaN,4,NaN,3,NaN,NaN,NaN,12,NaN,5,0,NaN,403.0,1,NaN,NaN,1,NaN,7,NaN,2,NaN,2,7,0,3,20,3,14,NaN,NaN,Wing Left,FINAL_TEAM,Wing,Midfield,NaN,NaN,2023.0,4.186815,4.886644,9.187695,-4.301051,-0.009818
28953,2023F4_Collingwood_BrisbaneLions,Collingwood,Tom Mitchell,2023F4,Tom_Mitchell,MIDFIELDER,CD_I291856,30,182,88,6,LEFT,WA,2011.0,2013.0,Ashburton (Vic)/Claremont (WAFL)/Sydney/Hawthorn,21.0,fatherSon,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-05-31,72,0,0.0,NaN,1,2,NaN,NaN,NaN,0,NaN,10,NaN,NaN,NaN,NaN,87.5,24,124,NaN,NaN,NaN,0,1,0.0,2,0,NaN,13,0,NaN,NaN,NaN,3,NaN,3,NaN,NaN,NaN,11,NaN,4,0,NaN,246.0,1,NaN,NaN,1,NaN,7,NaN,0,NaN,6,13,1,7,25,2,15,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023.0,0.000000,12.433041,12.081497,0.351545,7.074374
28954,2023F4_Collingwood_BrisbaneLions,Collingwood,Will Hoskin-Elliott,2023F4,Will_Hoskin-Elliott,MIDFIELDER_FORWARD,CD_I291720,30,186,82,32,RIGHT,VIC,2011.0,2012.0,North Sunshine (Vic)/Western U18/GWS,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-09-02,89,0,0.0,NaN

In [9]:
exp_vaep_player_stats_output_path

'/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v4.csv'

In [10]:
player_stats.to_csv(exp_vaep_player_stats_output_path, index=False)

In [ ]:
predict_player_stats(scored_chains = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_match_chains_v4.csv", 
                     player_stats_file_path = "/Users/ciaran/Documents/Projects/AFL/data/player_stats.csv", 
                     scored_player_stats_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v4.csv")

In [ ]:
scored_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v4.csv")
scored_data.head()

In [ ]:
scored_data.tail()